**Chapter 3: Getting Started with LangChain**

In [8]:
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)

from config import set_environment
set_environment()

## Exploring API model integrations

### Google Cloud Platform

In [12]:
import vertexai
PROJECT_ID = "elevated-epoch-417712"
REGION = "us-central1"
vertexai.init(project=PROJECT_ID, location=REGION)

In [13]:
from langchain.llms import VertexAI
from langchain import PromptTemplate, LLMChain

template = """Question: {question}
Answer: Let's think step by step."""
prompt = PromptTemplate(template=template, input_variables=["question"])
llm = VertexAI()
llm_chain = LLMChain(prompt=prompt, llm=llm, verbose=True)
question = "What NFL team won the Super Bowl in the year Justin Beiber was born?"
llm_chain.run(question)



> Entering new LLMChain chain...
Prompt after formatting:
Question: What NFL team won the Super Bowl in the year Justin Beiber was born?
Answer: Let's think step by step.

> Finished chain.


' Justin Bieber was born on March 1, 1994. The Super Bowl is held in February, so the Super Bowl that happened in the year Justin Bieber was born would have been Super Bowl XXVIII, which was held on January 30, 1994. The Dallas Cowboys won Super Bowl XXVIII.\n\nThe final answer is Dallas Cowboys'

## Building an application for customer service

### Text classification

In [14]:
from huggingface_hub import list_models

def list_most_popular(task: str):
    for rank, model in enumerate(list_models(filter=task, sort="downloads", direction=-1)):
        if rank == 5:
            break
        print(f"{model.id}, {model.downloads}\n")
        
list_most_popular("text-classification")

cardiffnlp/twitter-roberta-base-sentiment-latest, 132872180

mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis, 25250863

smilegate-ai/kor_unsmile, 10375883

lxyuan/distilbert-base-multilingual-cased-sentiments-student, 7789676

cardiffnlp/twitter-roberta-base-irony, 7522276



In [15]:
from transformers import pipeline

customer_email = """
I am writing to pour my heart out about the recent unfortunate experience
I had with one of your coffee machines that arrived broken. I anxiously
unwrapped the box containing my highly anticipated coffee machine.
However, what I discovered within broke not only my spirit but also any
semblance of confidence I had placed in your brand.
Its once elegant exterior was marred by the scars of travel, resembling a
war-torn soldier who had fought valiantly on the fields of some espresso
battlefield. This heartbreaking display of negligence shattered my dreams
of indulging in daily coffee perfection, leaving me emotionally distraught
and inconsolable
"""

sentiment_model = pipeline(
    task="sentiment-analysis",
    model="cardiffnlp/twitter-roberta-base-sentiment"
)
print(sentiment_model(customer_email))

[{'label': 'LABEL_0', 'score': 0.7691405415534973}]


### Summarization

In [16]:
list_most_popular("summarization")

google-t5/t5-small, 3657071

google-t5/t5-base, 2734080

facebook/bart-large-cnn, 2351012

philschmid/bart-large-cnn-samsum, 590044

sshleifer/distilbart-cnn-12-6, 502373



In [17]:
from langchain import HuggingFaceHub

summarizer = HuggingFaceHub(
    repo_id="facebook/bart-large-cnn",
    model_kwargs={"temperature": 0, "max_length": 180}
)

def summarize(llm, text) -> str:
    return llm(f"Summarize this: {text}!")
    
summarize(summarizer, customer_email)

'A customer\'s coffee machine arrived broken. "This heartbreaking display of negligence shattered my dreams," writes the customer. "I was emotionally distraught and inconsolable," he adds. "It was like a war-torn soldier who had fought valiantly on the fields of some espressobattlefield"'

In [18]:
from langchain.llms import VertexAI
from langchain import PromptTemplate, LLMChain

template = """Given this text, decide what is the issue the customer is
concerned about. Valid categories are these:
* product issues
* delivery problems
* missing or late orders
* wrong product
* cancellation request
* refund or exchange
* bad support experience
* no clear reason to be upset
Text: {email}
Category:
"""

prompt = PromptTemplate(template=template, input_variables=["email"])
llm = VertexAI()
llm_chain = LLMChain(prompt=prompt, llm=llm, verbose=True)
print(llm_chain.run(customer_email))



> Entering new LLMChain chain...
Prompt after formatting:
Given this text, decide what is the issue the customer is
concerned about. Valid categories are these:
* product issues
* delivery problems
* missing or late orders
* wrong product
* cancellation request
* refund or exchange
* bad support experience
* no clear reason to be upset
Text: 
I am writing to pour my heart out about the recent unfortunate experience
I had with one of your coffee machines that arrived broken. I anxiously
unwrapped the box containing my highly anticipated coffee machine.
However, what I discovered within broke not only my spirit but also any
semblance of confidence I had placed in your brand.
Its once elegant exterior was marred by the scars of travel, resembling a
war-torn soldier who had fought valiantly on the fields of some espresso
battlefield. This heartbreaking display of negligence shattered my dreams
of indulging in daily coffee perfection, leaving me emotionally distraught
and inconsolable

Ca